# Inverse Schemes for Geodynamic Applications

## Introduction

In this tutorial, we will explore the use of adjoint schemes in geodynamics. Adjoint schemes are a powerful tool for solving inverse problems, allowing us to retrieve unknown parameters or initial conditions in physical systems.

The exercise involves reconstructing the initial temperature field of a single upwelling plume in an enclosed square computational domain. The problem assumes isoviscous, incompressible flow under the Boussinesq approximation, with a Rayleigh number of $Ra=10^6$.

## Setting Up Dependencies

Firstly, let's make sure we have all the dependencies installed. This will set up Firedrake and other required libraries on this Google Colab environment.

In [ ]:
# This magic makes plots appear 
%matplotlib inline
import matplotlib.pyplot as plt
import pyvista as pv

import os
os.environ["GADOPT_LOGLEVEL"] = "WARNING"

# Load Firedrake on Colab
try:
    import firedrake
except ImportError:
    !wget "https://github.com/g-adopt/tutorials/releases/latest/download/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

## Problem Definition

In this exercise, we aim to reconstruct the initial temperature field of a single upwelling plume within an enclosed square computational domain. The domain has free-slip boundary conditions on all sides, assumes isoviscous rheology, incompressible flow under the Boussinesq approximation, and a Rayleigh number of \( Ra = 10^6 \).

- **Domain**: Enclosed square
- **Boundary Conditions**: Free-slip on all sides
- **Rheology**: Isoviscous
- **Flow Assumptions**: Incompressible, Boussinesq approximation
- **Rayleigh Number**: \( $Ra = 10^6$ \)


The model is heated from below ($ T_C = 1 $) and cooled from above ($ T_S = 0 $). We will run the model for 80 time steps with $\delta t = 4 \times 10^{-6}$ to allow the temperature anomaly to form a plume that reaches the domain's top boundary.


In [ ]:
# Clear the tape of any previous operations to ensure
# the adjoint reflects the forward problem we solve here
tape = get_working_tape()
tape.clear_tape()

# Rayleigh number
Ra = Constant(1e6)

# Time stepping parameters
max_timesteps = 80
delta_t = Constant(4e-6)

# Boundary tags
bottom_id, top_id, left_id, right_id = "bottom", "top", 1, 2

## Mesh and Function Spaces

The computational domain is a unit square with 150 x 150 elements. We define function spaces for velocity, pressure, and temperature within this mesh.

1. **Vector Function Space (V)**: This is used for the velocity field and employs a \(Q2\) discretization.
2. **Scalar Function Space (W)**: This is used for pressure and uses a \(Q1\) discretization.
3. **Mixed Function Space (Z)**: This combines the velocity and pressure function spaces.
4. **Scalar Function Space (Q)**: This is used for the temperature field and also uses a \(Q2\) discretization.


In [ ]:
# Load the mesh
with CheckpointFile("mesh.h5", "r") as f:
    mesh = f.load_mesh("firedrake_default_extruded")

# Define function spaces
V = VectorFunctionSpace(mesh, "CG", 2)  # Velocity function space (vector)
W = FunctionSpace(mesh, "CG", 1)  # Pressure function space (scalar)
Q = FunctionSpace(mesh, "CG", 2)  # Temperature function space (scalar)
Z = MixedFunctionSpace([V, W])  # Mixed function space

## Initial Conditions and Boundary Conditions

The initial temperature field is generated by a Gaussian anomaly superimposed on an average temperature profile, which we already covered in a previous tutorial (`06-GD-2D-convection.ipynb`).

For velocity, free-slip boundary conditions are applied on all sides of the domain.

Boundary tags are used to identify different boundaries:
- Tag 1: Plane \( x=0 \) (left)
- Tag 2: Plane \( x=1 \) (right)
- Tag "bottom": \( y=0 \)
- Tag "top": \( y=1 \)


In [ ]:
# Initialize the control variable for temperature
Tic = Function(Q1, name="Initial Temperature")

# Load from checkpoint
checkpoint_file = CheckpointFile("Checkpoint_State.h5", "r")
Tic.project(checkpoint_file.load_function(mesh, "Temperature", idx=max_timesteps - 1))

# Boundary conditions
stokes_bcs = {
    bottom_id: {"uy": 0},
    top_id: {"uy": 0},
    left_id: {"ux": 0},
    right_id: {"ux": 0},
}
temp_bcs = {
    bottom_id: {"T": 1.0},
    top_id: {"T": 0.0},
}

## Solvers and Time-stepping

To solve the problem, we use two main solvers:

1. **Energy Solver (`energy_solver`)**: This solver handles the energy equation. It takes in the temperature field \( T \), velocity \( u \), the physical approximation, time step \( \delta t \), and boundary conditions.

2. **Stokes Solver (`stokes_solver`)**: This solver handles the Stokes equation for velocity and pressure. It takes in the mixed function \( z \) (which contains both velocity and pressure), the temperature \( T \), the physical approximation, and boundary conditions.

The model is run for 80 time steps of \( \delta t = 4 \times 10^{-6} \) to capture the evolution of the system.


In [ ]:
# Enable disk checkpointing
enable_disk_checkpointing()

# Create the solvers
energy_solver = EnergySolver(
    T,
    u,
    approximation,
    delta_t,
    ImplicitMidpoint,
    bcs=temp_bcs,
)

stokes_solver = StokesSolver(
    z,
    T,
    approximation,
    bcs=stokes_bcs,
    nullspace=Z_nullspace,
    transpose_nullspace=Z_nullspace,
)

# Time-stepping loop
for timestep in range(0, max_timesteps):
    stokes_solver.solve()
    energy_solver.solve()
    # ... (Additional code for checkpointing and misfit calculations)


## Objective Functional and Optimization

The objective functional is composed of several terms:

1. **Temperature Misfit** ($ t_{\text{misfit}} $): Difference between the final temperature field and observations.
2. **Velocity Misfit** ($ u_{\text{misfit}} $): Accumulated surface velocity misfit using observed values.
3. **Damping Term** ($ \text{damping} $): Damping based on the difference between the initial condition and the average temperature profile.
4. **Smoothing Term** ($ \text{smoothing} $): Smoothing applied to the initial condition.

We then use adjoint-based optimization to minimize this functional, subject to constraints on the temperature field.



In [ ]:
# Define the objective functional components
t_misfit = assemble((T - Tobs) ** 2 * dx)
u_misfit = assemble(dot(u - uobs, u - uobs) * ds_t)
damping = assemble((Tic - Taverage) ** 2 * dx)
smoothing = assemble(dot(grad(Tic - Taverage), grad(Tic - Taverage)) * dx)

# Combine into the objective functional
objective = (
    t_misfit +
    alpha_u * (norm_obs * u_misfit / max_timesteps / norm_u_surface) +
    alpha_d * (norm_obs * damping / norm_damping) +
    alpha_s * (norm_obs * smoothing / norm_smoothing)
)

# Define the reduced functional and optimization problem
control = Control(Tic)
reduced_functional = ReducedFunctional(objective, control)

# ... (Optimization code)

## Conclusion

In this workshop, we've explored the fundamentals of adjoint-based optimization in geodynamics using the G-ADOPT package and Firedrake. We set up a problem that aimed to retrieve the initial temperature field of a single upwelling plume in an enclosed square domain. 

Through the use of various solvers and optimization techniques, we were able to gain insights into the inversion process and the relevant geophysical parameters. 

### Key Takeaways
1. Adjoint schemes provide a powerful tool for solving inverse problems in geodynamics.
2. The choice of function spaces and solvers can significantly impact the quality of the inversion.
3. Regularization terms like damping and smoothing are essential for a well-posed inverse problem.

### Further Steps
- Experiment with different initial conditions and boundary conditions.
- Explore the impact of varying Rayleigh numbers on the inversion process.
- Investigate more complex rheological models.



: 

## References and Additional Resources

In [ ]:
# Placeholder code